In [1]:
import pandas as pd 
import os

# testデータの読み込み
test_file_path = "/apartments_price/test/original_data/test.csv"
df = pd.read_csv(test_file_path, index_col=0, low_memory=False, encoding="utf-8")


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19466 entries, 1000000 to 47007421
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   種類          19466 non-null  object 
 1   地域          0 non-null      float64
 2   市区町村コード     19466 non-null  int64  
 3   都道府県名       19466 non-null  object 
 4   市区町村名       19466 non-null  object 
 5   地区名         19463 non-null  object 
 6   最寄駅：名称      19453 non-null  object 
 7   最寄駅：距離（分）   19453 non-null  object 
 8   間取り         18544 non-null  object 
 9   面積（㎡）       19466 non-null  int64  
 10  土地の形状       0 non-null      float64
 11  間口          0 non-null      float64
 12  延床面積（㎡）     0 non-null      float64
 13  建築年         18804 non-null  object 
 14  建物の構造       18201 non-null  object 
 15  用途          13480 non-null  object 
 16  今後の利用目的     18439 non-null  object 
 17  前面道路：方位     0 non-null      float64
 18  前面道路：種類     0 non-null      float64
 19  前面道路：幅員（ｍ）  0 non-nul

In [3]:
df.head()

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
ID,,,,,,,,,,,,,,,,,,,,,
1000000,中古マンション等,NaN,1101,北海道,札幌市中央区,旭ケ丘,円山公園,26,３ＬＤＫ,75,...,住宅,NaN,NaN,NaN,第１種低層住居専用地域,40.0,60.0,2020年第２四半期,未改装,NaN
1000056,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１１丁目,1,２ＬＤＫ,55,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2020年第１四半期,未改装,NaN
1000108,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１Ｒ,15,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第２四半期,未改装,NaN
1000109,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＬＤＫ,45,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第２四半期,改装済,NaN
1000110,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,3,１Ｒ,20,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第２四半期,NaN,NaN


## テストデータの前処理

In [19]:
import pandas as pd
import numpy as np
import os

# categorical_features の定義
categorical_features = [
    "都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造",
    "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"
]

# 前処理関数の定義
def data_pre(df):
    nonnull_list = [col for col in df.columns if df[col].count() == 0]
    df = df.drop(nonnull_list, axis=1)

    unnecessary_columns = ["市区町村名", "種類"]
    for col in unnecessary_columns:
        if col in df.columns:
            df = df.drop(col, axis=1)

    dis = {
        "30分?60分": 45,
        "1H?1H30": 75,
        "2H?": 120,
        "1H30?2H": 105,
    }
    if "最寄駅：距離（分）" in df.columns:
        df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    if "面積（㎡）" in df.columns:
        df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

    if "建築年" in df.columns:
        df["建築年"] = df["建築年"].astype(str)
        y_list = {}
        for i in df["建築年"].value_counts().keys():
            if "平成" in i:
                num = float(i.split("平成")[1].split("年")[0])
                year = 35 - num
            elif "令和" in i:
                num = float(i.split("令和")[1].split("年")[0])
                year = 3 - num
            elif "昭和" in i:
                num = float(i.split("昭和")[1].split("年")[0])
                year = 96 - num
            elif "戦前" in i:
                year = 76.0
            else:
                year = np.nan
            y_list[i] = year
        df["建築年"] = df["建築年"].replace(y_list).astype(float)

    if "取引時点" in df.columns:
        year = {
            "年第１四半期": ".25",
            "年第２四半期": ".50",
            "年第３四半期": ".75",
            "年第４四半期": ".99"
        }
        for k, j in year.items():
            df["取引時点"] = df["取引時点"].str.replace(k, j, regex=False)
        df["取引時点"] = df["取引時点"].astype(float)

    for col in categorical_features:
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df

# testデータの読み込みと前処理
test_file_path = "/Users/hayakawakazue/Downloads/Nishika/apartments_price/test/original_data/test.csv"
df = pd.read_csv(test_file_path, index_col=0, low_memory=False, encoding="utf-8")
df_processed = data_pre(df)

# **インデックスからIDを列に戻す**
df_processed = df_processed.reset_index()  # IDを列に戻す

# 前処理後のデータを保存
save_dir = '/Users/hayakawakazue/Downloads/Nishika/apartments_price/test/original_data'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

df_path = os.path.join(save_dir, 'test_proceeded.csv')
df_processed.to_csv(df_path, index=False)  # index=Falseでインデックスを含めない

print(f"データを保存しました: {df_path}")

# カテゴリカル列のインデックス確認（テストデータ）
cat_feature_indices_test = [df_processed.columns.get_loc(col) for col in categorical_features if col in df_processed.columns]
print("カテゴリカル変数のインデックス（テストデータ）:", cat_feature_indices_test)


データを保存しました: /Users/hayakawakazue/Downloads/Nishika/apartments_price/test/original_data/test_proceeded.csv
カテゴリカル変数のインデックス（テストデータ）: [2, 3, 4, 6, 9, 10, 11, 12, 16, 17]


/var/folders/ys/5xpvjn3d5p96rvqpz31hvh380000gn/T/ipykernel_3507/3169873629.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["建築年"] = df["建築年"].replace(y_list).astype(float)


In [20]:
# データ前処理の関数を適用
df_processed = data_pre(df)

# DataFrameの情報を確認
df_processed.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19466 entries, 1000000 to 47007421
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   市区町村コード    19466 non-null  int64   
 1   都道府県名      19466 non-null  category
 2   地区名        19463 non-null  category
 3   最寄駅：名称     19453 non-null  category
 4   最寄駅：距離（分）  19453 non-null  float64 
 5   間取り        18544 non-null  category
 6   面積（㎡）      19466 non-null  float64 
 7   建築年        18804 non-null  float64 
 8   建物の構造      18201 non-null  category
 9   用途         13480 non-null  category
 10  今後の利用目的    18439 non-null  category
 11  都市計画       19122 non-null  category
 12  建ぺい率（％）    19045 non-null  float64 
 13  容積率（％）     19045 non-null  float64 
 14  取引時点       19466 non-null  float64 
 15  改装         17036 non-null  category
 16  取引の事情等     152 non-null    category
dtypes: category(10), float64(6), int64(1)
memory usage: 1.7 MB


/var/folders/ys/5xpvjn3d5p96rvqpz31hvh380000gn/T/ipykernel_3507/3169873629.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["建築年"] = df["建築年"].replace(y_list).astype(float)
